In [55]:
from keras_wc_embd import WordCharEmbd
import numpy as np
import keras
import pandas as pd

sentences = [
    ['All','i','want','to','fuck','shit'],
    ['I','am','Pretty','Good'],
    ['fuck','this','shit'],
]


## Output Target Column

In [59]:
df2 = pd.read_excel('../dataset.xlsx') #Excelden oku

In [70]:
targetColumn = list(df2['TANILAR'])
targetCorpus = []
for i in targetColumn:
    row = i.split(", ")
    for c in row:
        if c not in targetCorpus:
            targetCorpus.append(c)
targetCorpus = np.array(targetCorpus)

1617

In [76]:
y_data = []
for i in targetColumn:
    row = i.split(", ")
    actualRow=[]
    for q in targetCorpus:
        if q not in row:
            actualRow.append(0.0)
        else:
             actualRow.append(1.0)
    y_data.append(actualRow)
y_data = np.array(y_data)



In [77]:
len(y_data)

1617

In [78]:
df = pd.read_excel('../dataset.xlsx') #Excelden oku

In [79]:
sikayet = df['SIKAYET'].apply(str)

In [80]:
sentences = []
for cumle in sikayet:
    sentences.append(cumle.split())

In [82]:
wc_embd = WordCharEmbd(
    word_min_freq=0,
    char_min_freq=0,
    word_ignore_case=False,
    char_ignore_case=False,
)
for sentence in sentences:
    wc_embd.update_dicts(sentence)

In [ ]:
wc_embd.get_batch_input(sentences)

In [98]:
np.asarray(y_data)

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 1.]])

In [112]:


inputs, embd_layer = wc_embd.get_embedding_layer()
lstm_layer = keras.layers.LSTM(units=5, name='LSTM')(embd_layer)
softmax_layer = keras.layers.Dense(units=len(y_data[0]), activation='softmax', name='Softmax')(lstm_layer)
model = keras.models.Model(inputs=inputs, outputs=softmax_layer)
model.compile(
    optimizer='adam',
    loss=keras.losses.categorical_crossentropy,
    metrics=[keras.metrics.categorical_accuracy],
)
model.summary()


def batch_generator():
    while True:
        yield wc_embd.get_batch_input(sentences), np.asarray(y_data)



Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_Char (InputLayer)         [(None, None, 19)]   0                                            
__________________________________________________________________________________________________
Input_Word (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
Embedding_Char_Pre (Embedding)  (None, None, 19, 30) 2010        Input_Char[0][0]                 
__________________________________________________________________________________________________
Embedding_Word (Embedding)      (None, None, 300)    626700      Input_Word[0][0]                 
___________________________________________________________________________________________

In [ ]:
model.fit(
    wc_embd.get_batch_input(sentences),
   [np.asarray(y_data)]
)

In [ ]:
model.fit_generator(
    generator=batch_generator(),
    steps_per_epoch=200,
    epochs=1,
)

  3/200 [..............................] - ETA: 3:18:06 - loss: 13.5926 - categorical_accuracy: 0.0239

In [92]:
model.predict(
   wc_embd.get_batch_input(['firat','dogan','shit','Good']), batch_size=None, verbose=0, steps=None, callbacks=None, max_queue_size=10,
    workers=1, use_multiprocessing=False
)

array([[0.49338186, 0.50661814],
       [0.49584606, 0.50415397],
       [0.4919908 , 0.50800914],
       [0.501754  , 0.49824604]], dtype=float32)

In [ ]:
wc_embd.get_word_dict()

In [ ]:
wc_embd.get_dicts()